# Weather Forecast Bias Correction using Deep Learning

This notebook implements a hybrid deep learning approach for weather forecast bias correction, combining:
1. LSTM for temporal pattern learning
2. Graph Neural Network for spatial relationships
3. Attention mechanism for feature fusion
4. Monte Carlo dropout for uncertainty estimation

## Setup and Installation

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone the repo if not already done
!rm -rf weather-bias-correction-dl
!git clone [https://github.com/MaheshSharan/weather-bias-correction-dl.git](https://github.com/MaheshSharan/weather-bias-correction-dl.git)

# Change to project directory
%cd weather-bias-correction-dl

# Uninstall any existing installation
!pip uninstall -y weather_bias_correction

# Install dependencies and package in editable mode
!pip install -r requirements.txt
!pip install -e .

# Setup project paths
import os
import sys

# Add the project directory to Python path
PROJECT_DIR = os.getcwd()
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

# Create necessary directories
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
MODELS_DIR = os.path.join(PROJECT_DIR, 'models')
LOGS_DIR = os.path.join(PROJECT_DIR, 'logs')

for dir_path in [DATA_DIR, MODELS_DIR, LOGS_DIR]:
    os.makedirs(dir_path, exist_ok=True)

# Verify the imports work
from src.data.download_ncep import NCEPDownloader
from src.data.download_gsod import GSODDownloader
from src.data.data_alignment import DataAligner

## Data Download and Preprocessing

In [1]:

from src.data.download_gsod import GSODDownloader
from src.data.data_alignment import DataAligner

from src.data.simple_openmeteo import SimpleOpenMeteoDownloader

# And update the downloader initialization:
openmeteo_downloader = SimpleOpenMeteoDownloader(
    output_dir=os.path.join(DATA_DIR, 'raw', 'openmeteo'),
    start_date='2018-01-01',
    end_date='2023-12-31'
)
gsod_downloader = GSODDownloader(
    output_dir=os.path.join(DATA_DIR, 'raw', 'gsod'),
    start_date='2018-01-01',
    end_date='2023-12-31'
)

# Example locations
locations = [
    {"name": "Berlin", "lat": 52.52, "lon": 13.41, "gsod_station": "10384"},
    {"name": "New_York", "lat": 40.71, "lon": -74.01, "gsod_station": "72506"},
    {"name": "Tokyo", "lat": 35.68, "lon": 139.77, "gsod_station": "47662"}
]

# Download data
print("Downloading Open-Meteo data...")
for location in locations:
    print(f"\nDownloading data for {location['name']}...")
    openmeteo_file = openmeteo_downloader.download_data(
        latitude=location['lat'],
        longitude=location['lon'],
        location_name=location['name']
    )

print("\nDownloading GSOD data...")
for location in locations:
    print(f"\nDownloading GSOD data for {location['name']}...")
    gsod_file = gsod_downloader.download_data(
        station_id=location['gsod_station']
    )

# Align data
print("\nAligning datasets...")
aligner = DataAligner(
    openmeteo_dir=os.path.join(DATA_DIR, 'raw', 'openmeteo'),
    gsod_dir=os.path.join(DATA_DIR, 'raw', 'gsod'),
    output_dir=os.path.join(DATA_DIR, 'processed')
)

# Align data for each location
for location in locations:
    print(f"\nAligning data for {location['name']}...")
    output_file = f"{location['name']}_2018-2023_aligned.csv"
    aligner.align_data(
        openmeteo_file=os.path.join(DATA_DIR, 'raw', 'openmeteo', f"{location['name']}_2018-01-01_2023-12-31.csv"),
        gsod_file=os.path.join(DATA_DIR, 'raw', 'gsod', f"{location['gsod_station']}.csv"),
        output_file=output_file
    )

ModuleNotFoundError: No module named 'src.data'

## Model Training

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from src.models.bias_correction_model import DeepBiasCorrectionModel
from src.training.train import train_model

# Set up model parameters
model_params = {
    'input_dim': 5,  # Number of input features
    'hidden_dim': 256,
    'num_layers': 3,
    'dropout_rate': 0.2,
    'learning_rate': 1e-3,
    'weight_decay': 1e-5
}

# Initialize model
model = DeepBiasCorrectionModel(**model_params)

# Set up callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath=MODELS_DIR,
    filename='bias_correction-{epoch:02d}-{val_loss:.2f}',
    monitor='val_loss',
    mode='min',
    save_top_k=3
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    mode='min'
)

# Set up logger
logger = TensorBoardLogger(LOGS_DIR, name='bias_correction')

# Train model
trainer = pl.Trainer(
    max_epochs=100,
    accelerator='gpu',
    devices=1,
    callbacks=[checkpoint_callback, early_stopping],
    logger=logger,
    gradient_clip_val=0.5
)

trainer.fit(model)

## Model Evaluation and Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from src.app.visualization import plot_predictions, plot_uncertainty

# Load best model
best_model = DeepBiasCorrectionModel.load_from_checkpoint(
    checkpoint_callback.best_model_path
)

# Make predictions with uncertainty
predictions, uncertainties = best_model.predict_with_uncertainty(test_data)

# Plot results
plot_predictions(test_data, predictions)
plot_uncertainty(test_data, predictions, uncertainties)

## Save Results to Drive

In [ ]:
# Save model and results
import torch
import pandas as pd

# Save model state
torch.save(best_model.state_dict(), os.path.join(MODELS_DIR, 'final_model.pth'))

# Save predictions
results_df = pd.DataFrame({
    'actual': test_data.y,
    'predicted': predictions,
    'uncertainty': uncertainties
})
results_df.to_csv(os.path.join(PROJECT_DIR, 'predictions.csv'))

print("Model and results saved to Google Drive!")